##  Autoencoding Simulated Curves 

In this notebook, we autoencode simulated growth curves leveraging out causal convolutional autoencoder architecture. We provide a full example of autoencoding simple growth curves (logistic) from scratch and then show the final results from a pre-trained model for the complex growth curves. The method of compressing these is identical to simple curves, but training takes longer due to larger training size. 

The code implementing the neural network architecture is provided in the `models` folder within this directory. We begin by importing necessarily numerical, scientific, and deep learning python libraries: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# For-loop visualization library
from tqdm.notebook import trange

# Deep learning
import torch
from torch.utils.data import DataLoader

# Configure GPU if available
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

display(device)

### Defining a Training Function

The first thing we do is define a generic training function which will take any autoencoder model and perform training, i.e. tuning model parameters to minimize the reconstruction loss via stochastic gradient descent. 

In [ ]:
def train(model, hp, data_loader, debug = False):
    """Train a given model for a specific type of hyperparamters and data_loader, train
    the model and then return the trained model as well as the running losses throughout 
    training.
    
    Parameters
    ----------
    model: torch.nn.Module
        An untrained model to be optimized.
    
    hp: dict
        A dictionary containing all of the hyperparameters for the system.
    
    data_loader: torch.utils.DataLoader
        A presetup dataloader containing the training data set for the set. 
    
    Returns
    -------
    model: torch.nn.Module
        The trained model after optimization.
        
    running_losses: list
        The loss for each epoch of training. 
    """
    
    # Store the losses per epoch
    running_losses = []
    
    # Configure optimizer. 
    optimizer = torch.optim.Adam(model.parameters(), lr=hp["lr"], weight_decay= hp["weight_decay"])

    criterion = torch.nn.MSELoss()
    
    # Outerloop will iterate through epochs. tqdm function trange provides progressbar
    for i in trange(hp["epochs"]):
        
        
        epoch_loss = 0 
        # Inner loop iterates through batches
        for batch in data_loader:

            # Transfer the batch to the GPU
            batch = batch.to(device)

            if debug:
                print("BATCH SHAPE: ")
                print(batch)

            # Zero gradient
            optimizer.zero_grad()

            # Perform forward pass
            pred, code = model(batch)

            # Uncomment to verify model prediction shape
            if debug:
                print("PREDI SHAPE: ")
                print(batch)

            # Compute reconstruction loss
            batch_loss = criterion(pred,batch)
            
            if debug:
                print(batch_loss)

            # Compute gradient
            batch_loss.backward()
            

            # Take step
            optimizer.step()

            # Append to running epoch loss
            epoch_loss += batch_loss.item()

        # Keep running track of losses
        if i % 250 == 0:
            print(f"Epoch [{i}]: " + str(epoch_loss))
    
        running_losses.append(epoch_loss)

    return model, running_losses

### Autoencoding Simple Growth Curves

To regenerate the results of figure 1B, we will now autoencode our previously generated simple growth curves. We begin by importing the dataset from the `saved_sims` folder and preparing it for training. 

In [ ]:
X = np.loadtxt("./saved_sims/ex_simple.csv", delimiter = ",")

# 0-1 Normalize the dataset
X = (X - X.min())/(X.max() - X.min())

# Transfer it to torch Tensor
X = torch.Tensor(X)

# Reshape to match the dimensions of the encoder. 
X = X.reshape(( X.shape[0], 1, -1)).double()
print(X.shape)

#### Configuring the Model Hyperparameters

We now import our NN model and configure the hyperparameters, which are all specified in the dictionary `hp`. 

In [ ]:
hp = {
    "in_channels" : 1, # Number of convolutional channels, fixed to 1 for individual curves
    "channels": 5,  # Number of individual channels per layer, increase to boost model capacity
    "depth": 10, # Number of Causal layers in the encoder, increase to boost model capacity
    "reduced_size" : 2, # Size to shrink final embedding to 
    "out_channels" : 2, # ^
    "kernel_size": 3, # Size of convolutional channels, 3 is the smallest to incorporate local information per time point
    "window_length": X.shape[2], # Length of input time series
    "lr": 0.001,  # Learning rate for Adam optimizer
    "epochs": 1000, # Number of training iteractions
    "batch_size": 1000,  # Number of training examples in minibatch descent, maxes at training set size
    "weight_decay":0.0 # Amount of L2 regualirzation for parameters, by default set to zero. 
}

In [ ]:
from models.CausalAE import CausalAE

# Create a new model, increase precision to double floating point precision, transfer to GPU
model = CausalAE(hp)
model.double()
model.to(device);

We leverage the `pytorch` data loader class to handle our minibatch training method. 

In [ ]:
data_loader = DataLoader(X, batch_size = hp["batch_size"])

#### Train the Model

Lastly we perform training. We store both the optimized final model but also the running error throughout training. The training function will print off the current L2 error every 250 epochs.  On GPU, this training should take < 10 minutes for the default hyperparameters. 

In [ ]:
data_loader = DataLoader(X, batch_size = hp["batch_size"])

trained_model, running_losses = train(model, hp, data_loader)

In [ ]:
plt.title("Reconstruction Loss")
plt.ylabel("MSE")
plt.xlabel("Epoch")
plt.plot(running_losses)
plt.show()

#### Analyzing Simple Curve Autoencoding Results

Now we will probe the latent space of this fully trained model. We begin by pulling off trained encoder and decoder modules to enable downstream analysis. 

In [ ]:
# Grab the encoder and decoder
encoder = trained_model.encoder
decoder = trained_model.decoder

# Switch into evaluation mode
encoder.eval()
decoder.eval();

Now we generate and plot embeddings:

In [ ]:
# Embed the dataset, transfer the embeddings to cpu, eliminate the gradient, and push to numpy.
embeddings = encoder(X.to(device)).cpu().detach().numpy();
embeddings.shape

In [ ]:
plt.figure(figsize = (5,5))
plt.tick_params(
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,  
    left = False,
    right = False, # ticks along the top edge are off
    labelbottom=False,
    labelleft = False,
    zorder = 20) # labels along the bottom edge are off
plt.xlabel("Latent Dimension 1", labelpad= 20)
plt.ylabel("Latent Dimension 2", labelpad = 15)
plt.scatter(embeddings[:,0], embeddings[:,1], s = 0.2)
plt.show()

As we can see, the structure of the data embeds along a clearly 1 dimensional structure, consistent with the underlying 1 dimensional structure of the mechanistic model (varying only in $\mu$, the growth rate). 

We can also compare the original and reconstructed growth curves. 

In [ ]:
# Choose ten curves at random
subset = X[np.random.randint(0,X.shape[0], size = 5), :,:].to(device)

# Generate embeddings and reconstructions
embeddings = model.encoder(subset)
recons = model.decoder(embeddings).cpu().detach()

# Transfer to cpu and drop gradients to enable plotting
embeddings = embeddings.cpu().detach()
subset = subset.cpu().detach()

In [ ]:
# Plot initial, embeddings, and reconstructions
fig, axs = plt.subplots(1,3, figsize = (18,6))
axs = axs.flatten()

axs[0].plot(subset.squeeze().T, color = 'red')
axs[0].set_title("Originals")

axs[1].scatter(embeddings[:,0],embeddings[:,1], color = "red")
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].set_title("Embeddings")

axs[2].plot(recons.squeeze().T, color = 'red')
axs[2].set_title("Reconstructions")

plt.show()

## Embedding More Complex Growth Curves

We now can analyze the results from embedding more complex growth curves. Training can be performed with the same code as before, here we will analyze the results from pre-trained models with `E = 2` and `E = 10`.

#### Analyzing `E = 2` Embeddings

First we'll load in the dataset used to generate figures 1C and 1D. 

In [ ]:
X = np.loadtxt("./saved_sims/figure_simulations.csv", delimiter = ",")

# Reshape for embedding
X = X.reshape(X.shape[0], 1, X.shape[1])

Now we load in the model for 2D embedding of the dataset. 

In [ ]:
# Hyperparameters used for original training
hp = {
    "in_channels" : 1, 
    "channels": 10, 
    "depth": 10,
    "reduced_size" : 2,
    "out_channels" : 2, 
    "kernel_size": 3,
    "window_length":133,
    "lr": 1e-3, 
    "epochs": 1000,
    "batch_size": 300, 
    "weight_decay":0.0
}

In [ ]:
PATH =   "./saved_params/2dAE.pth"
trained_model = CausalAE(hp)
trained_model.load_state_dict(torch.load(PATH));

As before we plot the original dataset, the embeddings, and the reconstructions. 

In [ ]:
# Grab the encoder and decoder
encoder = trained_model.encoder.cpu()
decoder = trained_model.decoder.cpu()

# Switch into evaluation mode
encoder.eval()
decoder.eval();

In [ ]:
embeddings = encoder(torch.Tensor(X)).detach().numpy().T;
embeddings.shape

In 2 dimensions we plot the entire latent distribution:

In [ ]:
plt.figure(figsize = (5,5))
plt.xticks([])
plt.yticks([])
plt.scatter(embeddings[0,:], embeddings[1,:], s = 10)
plt.xlabel("Latent 1")
plt.ylabel("Latent 2")
plt.show()

As anticipated, the distribution no longer follows a simple one-dimensional shape, consistent with the underlying generative model no longer exhibiting a clear 1D structure.  We can also plot specific subset of the data:

In [ ]:
# Choose ten curves at random
subset = torch.Tensor(X[np.random.randint(0,X.shape[0], size = 5), :,:])

# Generate embeddings and reconstructions
embeddings_sub = encoder(subset)
recons = decoder(embeddings_sub).cpu().detach()

# Transfer to cpu and drop gradients to enable plotting
embeddings_sub = embeddings_sub.cpu().detach()
subset = subset.cpu().detach()

In [ ]:
# Plot initial, embeddings, and reconstructions
fig, axs = plt.subplots(1,3, figsize = (18,6))
axs = axs.flatten()

axs[0].plot(subset.squeeze().T, color = 'red')
axs[0].set_title("Originals")

axs[1].scatter(embeddings_sub[:,0],embeddings_sub[:,1], color = "red")
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].set_title("Embeddings")

axs[2].plot(recons.squeeze().T, color = 'red')
axs[2].set_title("Reconstructions")

plt.show()

Lastly we can also look at the principal component decomposition in the latent space to analyze the structure of the learned embeddings. 

In [ ]:
from sklearn.decomposition import PCA

# Compute a PCA decomposition of the dataset
pca = PCA()
X_pca = pca.fit_transform(embeddings.T)

In [ ]:
# Plot the variance explaiend by each principal component

variances = pca.explained_variance_ratio_

# Plot formatting
plt.figure(figsize = (4,4))
plt.bar([f"{x+1}" for x in range(variances.shape[0])],height = variances)
plt.ylim(0,1.0)
plt.locator_params(axis='y', nbins=3)
plt.title("Variance Explained by Principal Component")
plt.show()

#### Analyzing `E = 10` Embeddings

We repeat the same analysis except for `E = 10` for the embedding dimension. 

In [ ]:
# Hyperparameters used for original training
hp = {
    "in_channels" : 1, 
    "channels": 10, 
    "depth": 10,
    "reduced_size" : 10,
    "out_channels" : 10, 
    "kernel_size": 3,
    "window_length":133,
    "lr": 1e-3, 
    "epochs": 1000,
    "batch_size": 300, 
    "weight_decay":0.0
}

In [ ]:
PATH =   "./saved_params/10dAE.pth"
trained_model = CausalAE(hp)
trained_model.load_state_dict(torch.load(PATH));

In [ ]:
# Grab the encoder and decoder
encoder = trained_model.encoder.cpu().double()
decoder = trained_model.decoder.cpu().double()

# Switch into evaluation mode
encoder.eval()
decoder.eval();

In [ ]:
embeddings = encoder(torch.Tensor(X).double())
embeddings.shape

In [ ]:
# Choose ten curves at random
subset = torch.Tensor(X[np.random.randint(0,X.shape[0], size = 5), :,:]).double()

# Generate embeddings and reconstructions
embeddings_sub = encoder(subset)
recons = decoder(embeddings_sub).cpu().detach()

# Transfer to cpu and drop gradients to enable plotting
embeddings_sub = embeddings_sub.cpu().detach()
subset = subset.cpu().detach()

In [ ]:
# Plot initial, embeddings, and reconstructions
fig, axs = plt.subplots(1,2, figsize = (18,6))
axs = axs.flatten()

axs[0].plot(subset.squeeze().T, color = 'red')
axs[0].set_title("Originals")

axs[1].plot(recons.squeeze().T, color = 'red')
axs[1].set_title("Reconstructions")

plt.show()

In [ ]:
from sklearn.decomposition import PCA

# Compute a PCA decomposition of the dataset
pca = PCA()
X_pca = pca.fit_transform(embeddings.detach().numpy())

In [ ]:
# Plot the variance explaiend by each principal component

variances = pca.explained_variance_ratio_

# Plot formatting
plt.figure(figsize = (4,4))
plt.bar([f"{x+1}" for x in range(variances.shape[0])],height = variances)
plt.ylim(0,1.0)
plt.locator_params(axis='y', nbins=3)
plt.title("Variance Explained by Principal Component")
plt.show()